# side model feature selection

In [1]:
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(15677, 3447)


adx_7                               0
mod_rsi_lag17                       0
mod_rsi_lag19                       0
mod_rsi_dt_lag1                     0
mod_rsi_dt_lag2                     0
                                   ..
comb_spectrum_dom_cycle_ddt_lag5    0
comb_spectrum_dom_cycle_ddt_lag6    0
comb_spectrum_dom_cycle_ddt_lag7    0
comb_spectrum_dom_cycle_ddt_lag8    0
williams_r_ddt_lag19                0
Length: 3447, dtype: int64

In [2]:
import numpy as np

label = np.load("data/side_label.npy")

print(label.shape)

len_gap = len(label) - len(df_features)

label = label[len_gap:]

print(label.shape)

(15741,)
(15677,)


In [12]:
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector()
selector.fit(df_features, label)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: williams_r (最大重要性: 0.0465)
➤ 计算特征冗余度...
➤ 总计选择689个特征 (已选择1个，还需选择688个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                           | 0/688 [00:00<?, ?特征/s]


✅ 特征选择完成：从3447个特征中选择了689个，舍弃了2758个
✅ 选择的特征: ['williams_r', 'adx_7_ddt', 'adaptive_stochastic_ddt', 'conv_0', 'vwap_dt', 'ehlers_early_onset_trend_ddt', 'cmma', 'ehlers_early_onset_trend_ddt_lag1', 'evenbetter_sinewave_short_dt', 'vwap_dt_lag1', 'fisher_dt', 'cmma_lag1', 'roofing_filter_ddt', 'stc_dt', 'norm_on_balance_volume', 'trendflex_dt', 'stc', 'williams_r_dt', 'acr', 'trendflex_dt_lag1', 'adx_14_dt', 'fisher_dt_lag1', 'acc_swing_index_dt', 'evenbetter_sinewave_long_dt', 'roofing_filter_dt', 'reactivity', 'voss_filt_dt', 'fisher_ddt', 'fisher_ddt_lag1', 'evenbetter_sinewave_short_dt_lag1', 'cmma_dt', 'iqr_ratio_dt', 'roofing_filter_ddt_lag1', 'williams_r_lag1', 'evenbetter_sinewave_short', 'stc_dt_lag1', 'adx_7_dt_lag1', 'voss', 'trendflex_ddt_lag1', 'natr_dt', 'aroon_diff', 'voss_dt_lag1', 'reactivity_dt', 'evenbetter_sinewave_long', 'adaptive_cci_dt', 'homodyne_ddt', 'ehlers_early_onset_trend_dt', 'stc_dt_lag2', 'highpass_bp_ddt_lag2', 'bandpass_dt', 'fisher', 'adaptive_stochas

In [15]:
side_res_long = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
print((side_res_long > 0).sum())
side_res_long

495


williams_r                      0.046539
cmma                            0.038528
vwap_dt                         0.032372
fisher_dt                       0.032203
evenbetter_sinewave_short_dt    0.030919
                                  ...   
conv_20                         0.000000
conv_21                         0.000000
conv_22                         0.000000
conv_23                         0.000000
williams_r_ddt_lag19            0.000000
Length: 3447, dtype: float64

In [17]:
import json

feature_info = {
    "side": {
        "side": [],
    },
    "meta": {
        "meta": [],
        "model": [],
    },
}

feature_long = side_res_long[side_res_long > 0].index.tolist()


feature_info["side"]["side"] = feature_long


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [2]:
import numpy as np
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")
print(df_features.shape)
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(15677, 3447)
(15677,)


adx_7                               0
mod_rsi_lag17                       0
mod_rsi_lag19                       0
mod_rsi_dt_lag1                     0
mod_rsi_dt_lag2                     0
                                   ..
comb_spectrum_dom_cycle_ddt_lag5    0
comb_spectrum_dom_cycle_ddt_lag6    0
comb_spectrum_dom_cycle_ddt_lag7    0
comb_spectrum_dom_cycle_ddt_lag8    0
williams_r_ddt_lag19                0
Length: 3447, dtype: int64

In [3]:
from strategies.BinanceBtcEntropyBarV1.config import (
    SIDE,
    get_side_model,
)

side_model = get_side_model(False)

side_res = side_model.predict(df_features[SIDE])

assert df_features.shape[0] == len(side_res)

df_features["model"] = side_res

In [4]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector()
selector.fit(df_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: williams_r (最大重要性: 0.0476)
➤ 计算特征冗余度...
➤ 总计选择689个特征 (已选择1个，还需选择688个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                           | 0/688 [00:00<?, ?特征/s]


✅ 特征选择完成：从3448个特征中选择了689个，舍弃了2759个
✅ 选择的特征: ['williams_r', 'adx_7_ddt', 'pfe_lag16', 'conv_0', 'cmma_ddt', 'corwin_schultz_estimator', 'trendflex_ddt', 'price_variance_ratio_ddt', 'bekker_parkinson_vol_dt', 'kyle_lambda_lag14', 'adx_14_ddt', 'model', 'fti_ddt', 'price_change_oscillator_dt', 'norm_on_balance_volume_ddt', 'ac_45', 'vwap_dt', 'amihud_lambda_dt', 'adx_7_dt', 'bekker_parkinson_vol', 'ac_1', 'acc_swing_index_dt', 'price_variance_ratio_dt', 'hurst_coef_fast_lag13', 'bekker_parkinson_vol_ddt', 'williams_r_ddt_lag16', 'fti_dt', 'kyle_lambda_dt', 'entropy_for_jesse_ddt_lag16', 'corwin_schultz_estimator_dt', 'hurst_coef_slow_ddt_lag19', 'price_change_oscillator_ddt_lag5', 'bandpass_dt', 'cmma_lag1', 'stc_ddt', 'natr_ddt_lag7', 'dft_spectrum_8', 'forecast_oscillator', 'fti_best_period_ddt_lag14', 'roll_measure_dt', 'bekker_parkinson_vol_ddt_lag1', 'forecast_oscillator_lag1', 'comb_spectrum_pwr_3', 'dft_dom_cycle_dt_lag3', 'price_variance_ratio_ddt_lag18', 'hasbrouck_lambda_lag9',

In [5]:
(meta_res > 0).sum()

877

In [6]:
import json
from pathlib import Path

feature_info = Path("strategies/BinanceBtcEntropyBarV1/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

feature_info["meta"]["meta"] = []
feature_info["meta"]["model"] = []

meta_features = meta_res[meta_res > 0].index.tolist()
for k in meta_features:
    if k == "model":
        feature_info["meta"]["model"].append(k)
    else:
        feature_info["meta"]["meta"].append(k)


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)